In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision
from torchvision import transforms

import os
import numpy as np
import random
from skimage import io
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from pandas import read_csv
from math import floor, ceil, sqrt, exp
from IPython import display
import time
from itertools import chain
import time
import warnings
from pprint import pprint
from PIL import Image
import torchvision.transforms.functional as TF

import sys
CURRENT_DIR = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.abspath(os.path.join(CURRENT_DIR, os.pardir))
sys.path.append(parent_dir)

from utils.helpers import crop_image
from models.efficientunet import CDUnet, UpSamplingBlock, ConvBlock

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
DATA_PATH = f"{CURRENT_DIR}/../datasets/concrete_cd_labeled-v2-2/concrete_cd_labeled-v2-2"

In [10]:
class ChangeDetectionTestDataset(Dataset):
    def __init__(self, data_path, transforms=None, mask_pattern=None):
        self.data_path = data_path
        self.transforms = transforms
        self.test_regex = None if mask_pattern is None else re.compile(mask_pattern) 
        self.weights = [1.0, 1.0]
        self.image_num = 0
        
        self.fetched_data = []
        self._fetch_paths(data_path)

    def _fetch_paths(self, data_path):
        total_pixels = 0.0
        positive_pixels = 0.0
        
        masks_path = os.path.join(data_path, "masks")
        matched_test_files = os.listdir(masks_path) if self.test_regex is None else [f for f in os.listdir(masks_path) if self.test_regex.match(f)]
        for mask_name in matched_test_files:
            if len(mask_name) >= 4 and mask_name[-4:] == ".PNG":
                video_name, snapshot_name = mask_name.split("_")
                snapshot_name = snapshot_name.split('.')[0]
                snapshots_dir_path = os.path.join(DATA_PATH, "data", video_name)
                before_patches = crop_image(np.array(Image.open(f"{snapshots_dir_path}/before_{snapshot_name}.png")))
                after_patches = crop_image(np.array(Image.open(f"{snapshots_dir_path}/after_{snapshot_name}.png")))
                
                uncropped_mask = np.array(Image.open(f"{masks_path}/{mask_name}"))[..., :1]
                
                total_pixels += np.prod(uncropped_mask.shape)
                positive_pixels += uncropped_mask.sum()
                mask_patches = crop_image(uncropped_mask)
                
                for before_sample, after_sample, mask_sample in zip(before_patches, after_patches, mask_patches):
                    self.fetched_data.append({"before":before_sample, "after":after_sample, "mask":mask_sample})
        self.image_num = len(matched_test_files)
        
    def augment_image_mask(
        self,
        patch_1: torch.Tensor,
        patch_2: torch.Tensor,
        mask: torch.Tensor, 
        probability: float = 0.8,
    ) -> tuple[torch.Tensor, torch.Tensor]:
       
        if random.random() <= probability:
            angle = random.randint(-50, 50)
            patch_1 = TF.rotate(patch_1, angle)
            patch_2 = TF.rotate(patch_2, angle)
            mask = TF.rotate(mask, angle)
        if random.random() <= probability:
            patch_1 = TF.vflip(patch_1)
            patch_2 = TF.vflip(patch_2)
            mask = TF.vflip(mask)  
        if random.random() <= probability:
            patch_1 = TF.hflip(patch_1)
            patch_2 = TF.hflip(patch_2)
            mask = TF.hflip(mask)

        return patch_1, patch_2, mask

    def __len__(self):
        return len(self.fetched_data)

    def __getitem__(self, idx):
        data_sample = self.fetched_data[idx]
        before_patch = data_sample["before"]
        after_patch = data_sample["after"]
        mask = data_sample["mask"]

        before_patch, after_patch, mask = TF.to_tensor(before_patch), TF.to_tensor(after_patch), TF.to_tensor(mask)
        before_patch, after_patch, mask = self.augment_image_mask(before_patch, after_patch, mask)
        before_patch = TF.normalize(before_patch, mean=(0.485), std=(0.229))
        after_patch = TF.normalize(after_patch, mean=(0.485), std=(0.229))

        return {"before":before_patch, "after":after_patch, "mask":mask}

In [11]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, probs, targets):
        probs = probs.view(-1)
        targets = targets.view(-1)

        intersection = (probs * targets).sum()
        dice_coeff = (2. * intersection + self.smooth) / (probs.sum() + targets.sum() + self.smooth)
        dice_loss = 1 - dice_coeff
        
        return dice_loss

loss_bce = nn.BCELoss()
loss_dice = DiceLoss()

In [12]:
dataset = ChangeDetectionTestDataset(DATA_PATH, mask_pattern=r"^(?!4A_|5A_).*")
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
resizer = torchvision.transforms.Resize([512, 512])
model = torch.load(f"{CURRENT_DIR}/../weights/semi_supervised_cracks/crack_segmentation.pth", map_location=device)
model.activate_threshold_mode()

In [ ]:
def initialize_population(size, thresholds_num, bounds):
    return [{f"threshold{i}": random.uniform(bounds[0], bounds[1]) for i in range(thresholds_num)} for _ in range(size)]

def predict_change(before_sample, after_sample):
    return model(before_sample, after_sample)

def evaluate_fitness(thresholds):
    model.eval()
    model.set_thresholds(thresholds)
    fitness_value = 0.0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            if i == 10: break
            before = resizer(batch["before"]).float().to(device)
            after = resizer(batch["after"]).float().to(device)
            mask = resizer(batch["mask"]).float().to(device)
            difference_map = predict_change(before, after)
            fitness_value += (loss_bce(difference_map, mask) + loss_dice(difference_map, mask)).item()
    return -fitness_value

def selection(population, fitnesses, tournament_size=3):
    selected_parents = []
    population_size = len(population)

    for _ in range(population_size):
        tournament_indices = random.sample(range(population_size), tournament_size)
        tournament_fitnesses = [fitnesses[i] for i in tournament_indices]
        
        best_index = tournament_indices[tournament_fitnesses.index(max(tournament_fitnesses))]
        selected_parents.append(population[best_index])
    
    return selected_parents

def crossover(parents, crossover_rate=0.7):
    offspring = []
    num_parents = len(parents)

    for i in range(0, num_parents, 2):
        parent1 = parents[i]
        parent2 = parents[i + 1] if i + 1 < num_parents else parents[0]
        
        child1, child2 = {}, {}
        for i in range(len(parent1)):
            if random.random() < crossover_rate:
                crossover_point = random.randint(1, len(parent1) - 1)
                C_min = min(parent1[f"threshold{i}"],  parent2[f"threshold{i}"])
                C_max = max(parent1[f"threshold{i}"],  parent2[f"threshold{i}"])
                I = C_max - C_min
                L = C_min - 0.15 * I
                U = C_max + 0.15 * I
                child1[f"threshold{i}"] = random.uniform(L, U)
                child2[f"threshold{i}"] = random.uniform(L, U)
            else:
                child1[f"threshold{i}"] = parent1[f"threshold{i}"]
                child2[f"threshold{i}"] = parent2[f"threshold{i}"]
        offspring.append(child1)
        offspring.append(child2)

    return offspring

def mutate(offsprings, mutation_rate, bounds):
    for offspring in offsprings:
        for chromosome in offspring.keys():
            if random.random() < mutation_rate:
                offspring[chromosome] = random.uniform(bounds[0], bounds[1])
    return offsprings

def select_new_generation(population, offspring, prev_fitnesses, retain=0.2, random_select=0.05):
    
    prev_fitnesses_pairs = list(zip(population ,prev_fitnesses))
    new_fitnesses_pairs = [(individual, evaluate_fitness(individual)) for individual in offspring]
    fitnesses = prev_fitnesses_pairs + new_fitnesses_pairs
    
    sorted_population = sorted(fitnesses, key=lambda x: x[1], reverse=True)
    
    retain_length = int(len(sorted_population) * retain)
    new_generation = [individual for individual, _ in sorted_population[:retain_length]]
    new_fitness = [fitness for _, fitness in sorted_population[:retain_length]]
    
    for individual, fitness in sorted_population[retain_length:]:
        if random_select > random.random():
            new_generation.append(individual)
            new_fitness.append(fitness)
    
    if len(new_generation) < len(population):
        for individual, fitness in sorted_population[:len(population) - len(new_generation)]:
            new_generation.append(individual)
            new_fitness.append(fitness)
    
    return new_generation[:len(population)], new_fitness[:len(population)]

In [ ]:
population_size = 30
thresholds_count = 6
bounds = (0, 5)  # Example bounds for thresholds
mutation_rate = 0.4
iterations_num = 50

population = initialize_population(population_size, thresholds_count, bounds)
population[0] = {"threshold0": 0.4, "threshold1": 0.6, "threshold2": 0.8,"threshold3": 1.0, "threshold4": 1.2, "threshold5": 1.4}
fitnesses = [evaluate_fitness(chromosome) for chromosome in population]

for i in range(iterations_num):
    print(i+1, max(fitnesses), fitnesses)
    selected_parents = selection(population, fitnesses)
    offspring = crossover(selected_parents)
    offspring = mutate(offspring, mutation_rate, bounds)
    population, fitnesses = select_new_generation(population, offspring, fitnesses)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    print(population[fitnesses.index(max(fitnesses))])
    print("\n")

KeyboardInterrupt: 

In [ ]:
population[fitnesses.index(max(fitnesses))]